In [1]:
# Modules that we need
from Vissim_env_class import environment, Load_Vissim
from MasterDQN_Agent import MasterDQN_Agent

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# General Libraries
import numpy as np 
import pylab as plt
import os
import csv
import pandas as pd
import json


In [ ]:
vissim \
= \
Load_Vissim(
Path_to_network = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\NSW_9_Cross_Experiment\\Single_Cross_TripleMOVA\\',\
inpx_Filename = 'Single_Cross_Triple.inpx',\
layx_Filename = 'Single_Cross_Triple.layx',\
attempts=5
)

# DQN Train

In [ ]:
model_name  = 'Single_Cross_Triple'
#vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
vissim_working_directory='C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\NSW_9_Cross_Experiment\\'

sim_length = 3601

agent_type = "DQN"
Session_ID = ""

# all controller actions
Single_Cross_Triple_dictionary8 =\
{\
    # Controller Number 0 
   'junctions' : {0 : {'default_actions' :  {    0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                             1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                             2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                             3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                                             4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                                             5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                                             6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                                             7 : [0,0,0,0,1,1,0,0,0,0,1,1]},
         
         'all_actions' :        {            0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                             1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                             2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                             3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                                             4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                                             5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                                             6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                                             7 : [0,0,0,0,1,1,0,0,0,0,1,1]},
 
         'link' : [1, 3, 5, 7],
         'lane' : ['1-1', '1-2', '1-3', '3-1', '3-2', '3-3', '5-1', '5-2', '5-3', '7-1', '7-2', '7-3'],
         
         'controled_by_com' : True,
         'agent_type' : agent_type,
         'green_time' : 6,
         'redamber_time' : 0,
         'amber_time' : 3, 
         'red_time' : 0,
         'state_size' : [13],
         'state_type' : 'QueuesSig',
         'reward_type' : 'Queues' ,
         'queues_counter_ID' : [1,2,3,4,5,6,7,8,9,10,11,12]             
         }
    },
   'demand' : { 'default' : [400, 400, 400, 400],
             
             0 : [300,300,300,300],
             1 : [600,600,600,600],
             2 : [1350,750,1350,750],
             3 : [1500,750,1500,750],
             4 : [1050,750,1050,750],
             5 : [750,1050,750,1050],
             6 : [750,1500,750,1500],
             7 : [750,1350,750,1350],
             8 : [600,600,600,600],
             9 : [300,300,300,300]
            }
}

In [ ]:
episodes = 400 
copy_weights_frequency = 20 # On a successfull run I copied the weight every 50

PER_activated = True
memory_size = 1000
batch_size = 128

gamma = 0.95
alpha = 0.005

In [ ]:
# Exploration Schedule ("linear" or "geometric")
exploration_schedule = "geometric"
epsilon_start = 1
epsilon_end   = 0.001

Random_Seed = 100

def choose_schedule(exploration_schedule, espilon_start, epsilon_end, episodes):
    if exploration_schedule == "linear":
        epsilon_decay = 1.2*(epsilon_end - epsilon_start)/(episodes-1)
        epsilon_sequence = [1 + epsilon_decay * entry for entry in range(episodes+1)]
        epsilon_sequence = [0 if entry < 0 else entry for entry in epsilon_sequence]
    elif exploration_schedule == "geometric":
        epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes-1)) # Geometric decay
        epsilon_sequence = [epsilon_start * epsilon_decay ** entry for entry in range(episodes+1)]
    elif exploration_schedule == "entropy":
        pass
    else:
        print("ERROR: Unrecognized choice of exploration schedule.")
        
    # Plotting exploration schedule
    plt.figure(figsize=(8,4.5))
    x_series = np.array(range(1,episodes+1))
    y_series = epsilon_sequence[0:episodes]
    plt.plot(x_series, y_series, '-b')
    plt.xlabel('Episodes')
    plt.ylabel('Ratio of random exploration')
    plt.title('Exploration schedule')
    plt.show()
    return(epsilon_sequence)

epsilon_sequence = choose_schedule(exploration_schedule, epsilon_start, epsilon_end, episodes)

In [ ]:
Single_Cross_Triple8_MultiDQN_Agents =\
MasterDQN_Agent(model_name, 
                vissim_working_directory, 
                sim_length, 
                Single_Cross_Triple_dictionary8,
                'default_actions',
                gamma, 
                alpha, 
                agent_type, 
                memory_size, 
                PER_activated, 
                batch_size, 
                copy_weights_frequency, 
                epsilon_sequence,
                Random_Seed = Random_Seed, 
                timesteps_per_second = 1, 
                Session_ID = Session_ID, 
                verbose = True)

In [ ]:
Single_Cross_Triple8_MultiDQN_Agents.prepopulate_memory(vissim = vissim)

In [ ]:
vissim = Single_Cross_Triple8_MultiDQN_Agents.env.Vissim

In [ ]:
#Single_Cross_Triple8_MultiDQN_Agents.train(400, vissim = vissim) 

In [ ]:
Single_Cross_Triple8_MultiDQN_Agents.load(400,best = True)

In [ ]:
Single_Cross_Straight_MultiDQN_Agents.demo(vissim=vissim)

# Debug

In [ ]:
self = Single_Cross_Triple8_MultiDQN_Agents
vissim_working_directory = self.vissim_working_directory 
model_name = self.model_name  
agent_type = self.agent_type
Session_ID = self.Session_ID
episode = 400
self.ID = 0 

Filename = os.path.join(vissim_working_directory, 
                        model_name, 
                        "Agents_Results", 
                        agent_type, 
                        Session_ID,
                        'Episode'+ str(episode) +'Agent'+str(self.ID)+'.h5')

In [ ]:
Filename

In [7]:
from tensorflow.keras.models import load_model

In [4]:
Filename = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\NSW_9_Cross_Experiment\\Single_Cross_Triple\\Agents_Results\\DQN\\BestAgent0.h5'

In [8]:
Filename = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\NSW_9_Cross_Experiment\\Single_Cross_Triple\\Agents_Results\\Single_Cross_Triple8_actions_DuelingDQN20c8\\BestAgent0.h5'

In [19]:
from tensorflow.keras.optimizers import Adam

model =load_model(Filename, compile=False)
model.compile(loss='mse', optimizer=Adam(lr=0.001))

In [16]:
load_model(Filename, custom_objects={
    'Adam': lambda **kwargs: hvd.DistributedOptimizer(keras.optimizers.Adam(**kwargs))
})

TypeError: Unexpected keyword argument passed to optimizer: name